In [ ]:
library(dplyr)
library(data.table)
library(tidyr)
library(lubridate)
library(ggplot2)

In [ ]:
ddg = fread(snakemake@input$preds) %>%
    select(MutationsID, Mutations,PDB,dEVOEF1,dPRODIGY,ddG) %>%
    mutate(seqname = strsplit(x = MutationsID, split = "|", fixed = T )) %>%
    select(-MutationsID) %>%
    unnest(cols = seqname)
nrow(ddg)

In [ ]:
df = fread(snakemake@input$additional_data)
ddg2 <- ddg %>%
    left_join(df, by = "seqname")
ddg2
    #left_join(ddg, by = "seqname")

In [ ]:
ddg4viz <- ddg2
ddg4viz$monthlabel <- ddg2$month
ddg4viz$month <- lubridate::parse_date_time(ddg2$month, "ym")
ddg4viz$month <- factor(ddg4viz$month, levels = sort(unique(ddg4viz$month)))

ddg4viz <- ddg4viz%>%
   pivot_longer(cols = c("dEVOEF1", "dPRODIGY", "ddG"), names_to = "Term", values_to = "ddG")

In [ ]:
names(ddg4viz)

In [ ]:
options(repr.plot.width=20, repr.plot.height=15)
p <- ggplot(data = ddg4viz) + geom_boxplot(aes (x = month, y = ddG)) +
    facet_wrap(~Term,  ncol=1, scales = "free") +
    theme(
        text = element_text(size = 20),
        axis.text.x = element_text(angle = 90)
    ) +
    scale_x_discrete(labels = abbreviate) +
    xlab("Month")
p

In [ ]:
ggsave(file = snakemake@output$png4rep, plot = p, dpi = 300, width = 20, height = 15)
fwrite(x = ddg2, file = snakemake@output$csv4rep )